In [2]:
#!/usr/bin/env python3
"""
External test for ElevenLabs webhook via NGINX proxy.
Tests the full production path: DNS -> NGINX -> Container -> Service
"""

import asyncio
import json
import sys
import time
import hmac
from hashlib import sha256

try:
    import httpx
except ImportError:
    print("ERROR: httpx not installed. Run: pip3 install httpx")
    sys.exit(1)


def generate_signature(payload_bytes: bytes, secret: str) -> str:
    """Generate HMAC signature for ElevenLabs webhook."""
    timestamp = int(time.time())
    payload_str = payload_bytes.decode("utf-8")
    full_payload = f"{timestamp}.{payload_str}"
    
    mac = hmac.new(
        key=secret.encode("utf-8"),
        msg=full_payload.encode("utf-8"),
        digestmod=sha256
    )
    
    return f"t={timestamp},v0={mac.hexdigest()}"


async def test_health_check(base_url: str):
    """Test health endpoint (no authentication required)."""
    print("\n=== Testing Health Check ===")
    print(f"URL: {base_url}/elevenlabs/health")
    
    async with httpx.AsyncClient(verify=False, timeout=10.0) as client:
        try:
            response = await client.get(f"{base_url}/elevenlabs/health")
            print(f"Status: {response.status_code}")
            print(f"Response: {response.text}")
            return response.status_code == 200
        except Exception as e:
            print(f"ERROR: {e}")
            return False


async def test_webhook_transcription(base_url: str, secret: str):
    """Test webhook endpoint with transcription payload."""
    print("\n=== Testing Webhook (Transcription) ===")
    print(f"URL: {base_url}/elevenlabs/webhook")
    
    # Sample transcription payload
    payload = {
        "type": "post_call_transcription",
        "conversation_id": "external_test_123",
        "agent_id": "agent_test_456",
        "data": {
            "conversation_id": "external_test_123",
            "agent_id": "agent_test_456",
            "status": "completed",
            "call_duration_secs": 45.5,
            "message_count": 4,
            "start_time_unix_secs": int(time.time()) - 60,
            "end_time_unix_secs": int(time.time()),
            "transcript": [
                {
                    "role": "agent",
                    "message": "Hello! How can I help you?",
                    "time_in_call_secs": 0.5
                },
                {
                    "role": "user",
                    "message": "I need information about my account.",
                    "time_in_call_secs": 3.0
                }
            ],
            "analysis": {
                "call_summary": "External test call - account inquiry",
                "evaluation": {
                    "sentiment": "neutral",
                    "issue_resolved": True
                }
            }
        }
    }
    
    payload_bytes = json.dumps(payload).encode("utf-8")
    signature = generate_signature(payload_bytes, secret)
    
    print(f"Payload size: {len(payload_bytes)} bytes")
    print(f"Signature: {signature[:50]}...")
    
    async with httpx.AsyncClient(verify=False, timeout=30.0) as client:
        try:
            response = await client.post(
                f"{base_url}/elevenlabs/webhook",
                content=payload_bytes,
                headers={
                    "elevenlabs-signature": signature,
                    "content-type": "application/json"
                }
            )
            print(f"Status: {response.status_code}")
            print(f"Response: {response.text}")
            
            if response.status_code == 200:
                print("✅ Webhook test PASSED")
                return True
            else:
                print("❌ Webhook test FAILED")
                return False
                
        except Exception as e:
            print(f"ERROR: {e}")
            return False


async def test_invalid_signature(base_url: str):
    """Test that invalid signatures are rejected."""
    print("\n=== Testing Invalid Signature (Should Fail) ===")
    
    payload = {"type": "post_call_transcription", "conversation_id": "test"}
    payload_bytes = json.dumps(payload).encode("utf-8")
    
    async with httpx.AsyncClient(verify=False, timeout=10.0) as client:
        try:
            response = await client.post(
                f"{base_url}/elevenlabs/webhook",
                content=payload_bytes,
                headers={
                    "elevenlabs-signature": "t=1234567890,v0=invalid_hash",
                    "content-type": "application/json"
                }
            )
            print(f"Status: {response.status_code}")
            print(f"Response: {response.text}")
            
            if response.status_code == 401:
                print("✅ Invalid signature correctly rejected")
                return True
            else:
                print("❌ Should have rejected invalid signature")
                return False
                
        except Exception as e:
            print(f"ERROR: {e}")
            return False


async def main():
    """Run all external tests."""
    print("=" * 60)
    print("ElevenLabs Webhook External Test Suite")
    print("=" * 60)
    
    # Configuration
    BASE_URL = "https://matrosmcp.duckdns.org"  # or http://92.5.238.158 if no SSL
    SECRET = "your-elevenlabs-webhook-secret-here"  # Must match ELEVENLABS_WEBHOOK_SECRET in .env
    
    print(f"\nBase URL: {BASE_URL}")
    print(f"Secret: {SECRET[:10]}..." if len(SECRET) > 10 else f"Secret: {SECRET}")
    print(f"\nNote: Testing from external IP will be blocked by NGINX IP whitelist")
    print(f"      unless your IP is in the ElevenLabs allowed list.\n")
    
    results = []
    
    # Test 1: Health check (no IP restriction)
    results.append(await test_health_check(BASE_URL))
    
    # Test 2: Valid webhook (will fail if IP not whitelisted)
    results.append(await test_webhook_transcription(BASE_URL, SECRET))
    
    # Test 3: Invalid signature
    results.append(await test_invalid_signature(BASE_URL))
    
    # Summary
    print("\n" + "=" * 60)
    print(f"Tests passed: {sum(results)}/{len(results)}")
    print("=" * 60)
    
    if all(results):
        print("✅ All tests PASSED")
    else:
        print("❌ Some tests FAILED")


# Run in Jupyter/IPython environment (already has event loop)
await main()

ElevenLabs Webhook External Test Suite

Base URL: https://matrosmcp.duckdns.org
Secret: your-eleve...

Note: Testing from external IP will be blocked by NGINX IP whitelist
      unless your IP is in the ElevenLabs allowed list.


=== Testing Health Check ===
URL: https://matrosmcp.duckdns.org/elevenlabs/health
ERROR: 

=== Testing Webhook (Transcription) ===
URL: https://matrosmcp.duckdns.org/elevenlabs/webhook
Payload size: 657 bytes
Signature: t=1767885351,v0=688a958c724b479d8574061207408d0f91...
Status: 200
Response: {"status":"received"}
✅ Webhook test PASSED

=== Testing Invalid Signature (Should Fail) ===
Status: 400
Response: {"detail":"Timestamp expired (533317481 seconds old)"}
❌ Should have rejected invalid signature

Tests passed: 1/3
❌ Some tests FAILED


In [5]:
TOPDESK_URL = "https://pietervanforeest-test.topdesk.net/tas/api"
TOPDESK_USERNAME = "api_aipilots"
TOPDESK_PASSWORD = "7w7j6-ytlqt-wpcbz-ywu6v-remw7"
import requests
import base64
import json

In [7]:
#create a sample ticket in TopDesk for testing purposes

import requests
import base64
import json

# TopDesk test instance configuration
TOPDESK_URL = "https://pietervanforeest-test.topdesk.net/tas/api"
TOPDESK_USERNAME = "api_aipilots"
TOPDESK_PASSWORD = "7w7j6-ytlqt-wpcbz-ywu6v-remw7"

# Create Basic Auth header
auth_token = base64.b64encode(f"{TOPDESK_USERNAME}:{TOPDESK_PASSWORD}".encode()).decode()
headers = {
    'Authorization': f'Basic {auth_token}',
    'Content-Type': 'application/json'
}

# Known valid caller ID (from TopDesk persons)
CALLER_ID = "d34b277f-e6a2-534c-a96b-23bf383cb4a1"  # Jacob Aalbregt

# Create test incident payload
incident_payload = {
    "briefDescription": "Test incident from Jupyter playground",
    "request": "This is a test incident created for testing purposes. It demonstrates the TopDesk API integration.",
    "caller": {
        "id": CALLER_ID
    },
    "category": {
        "name": "Core applicaties"
    },
    "priority": {
        "name": "P3 (I&A)"
    }
}

# Create the incident
response = requests.post(
    f"{TOPDESK_URL}/incidents",
    headers=headers,
    json=incident_payload,
    timeout=30
)

if response.status_code in [200, 201]:
    incident = response.json()
    print("✅ Incident created successfully!")
    print(f"   Incident Number: {incident.get('number')}")
    print(f"   Incident ID: {incident.get('id')}")
    print(f"   Caller: {incident.get('caller', {}).get('dynamicName')}")
    print(f"   Category: {incident.get('category', {}).get('name')}")
    print(f"   Priority: {incident.get('priority', {}).get('name')}")
    print(f"\n📋 Full response:")
    print(json.dumps(incident, indent=2))
else:
    print(f"❌ Failed to create incident")
    print(f"   Status: {response.status_code}")
    print(f"   Response: {response.text}")


✅ Incident created successfully!
   Incident Number: I2601 052
   Incident ID: a833a2d0-248c-48bd-9d3a-eadabe0cdd06
   Caller: Aalbregt, Jacob
   Category: Core applicaties
   Priority: P3 (I&A)

📋 Full response:
{
  "id": "a833a2d0-248c-48bd-9d3a-eadabe0cdd06",
  "status": "firstLine",
  "number": "I2601 052",
  "request": "08-01-2026 16:17 API_AIPilots,: \nThis is a test incident created for testing purposes. It demonstrates the TopDesk API integration.",
  "requests": "/tas/api/incidents/id/a833a2d0-248c-48bd-9d3a-eadabe0cdd06/requests",
  "action": "/tas/api/incidents/id/a833a2d0-248c-48bd-9d3a-eadabe0cdd06/actions",
  "attachments": "/tas/api/incidents/id/a833a2d0-248c-48bd-9d3a-eadabe0cdd06/attachments",
  "caller": {
    "id": "d34b277f-e6a2-534c-a96b-23bf383cb4a1",
    "dynamicName": "Aalbregt, Jacob",
    "phoneNumber": "015 515 5022",
    "mobileNumber": "0633638553",
    "email": "J.Aalbregt@pietervanforeest.nl",
    "branch": {
      "id": "979e1bf8-425a-4078-8c3a-2e8b87a1d

In [11]:
#request incident details from TopDesk for verification purposes

# Use the incident number or ID from the previous cell
# Replace this with the actual incident number you received
INCIDENT_NUMBER = "I2601 051"  # Example: "I 2412 001" or use incident['number'] from above

# Get incident details by number
response = requests.get(
    f"{TOPDESK_URL}/incidents/number/{INCIDENT_NUMBER}",
    headers=headers,
    timeout=30
)

if response.status_code == 200:
    incident_details = response.json()
    ticket_id = incident_details.get('id')
    
    print("✅ Incident details retrieved successfully!")
    print(f"\n📋 Incident: {incident_details.get('number')}")
    print(f"   ID: {ticket_id}")
    print(f"   Status: {incident_details.get('processingStatus', {}).get('name')}")
    print(f"   Brief: {incident_details.get('briefDescription')}")
    print(f"   Request: {incident_details.get('request')}")
    print(f"   Caller: {incident_details.get('caller', {}).get('dynamicName')}")
    print(f"   Category: {incident_details.get('category', {}).get('name')}")
    print(f"   Priority: {incident_details.get('priority', {}).get('name')}")
    print(f"   Created: {incident_details.get('creationDate')}")
    print(f"   Modified: {incident_details.get('modificationDate')}")
    
    # Check for action trail (transcripts are added as invisible actions)
    print("\n\n" + "=" * 80)
    print("📝 ACTION TRAIL (Call Transcripts)")
    print("=" * 80)
    
    actions_response = requests.get(
        f"{TOPDESK_URL}/incidents/id/{ticket_id}/actions",
        headers=headers,
        timeout=30
    )
    
    if actions_response.status_code == 200:
        actions = actions_response.json()
        print(f"\nRetrieved {len(actions)} action(s)\n")
        
        for i, action in enumerate(actions, 1):
            print(f"\n--- Action {i} ---")
            print(f"Entry Date: {action.get('entryDate')}")
            
            person = action.get('person')
            if person:
                print(f"Person: {person.get('name', 'N/A')}")
            else:
                print(f"Person: System/API")
            
            print(f"Invisible for Caller: {action.get('invisibleForCaller', False)}")
            
            memo_text = action.get('memoText', '')
            if memo_text:
                # Check if this is a call transcript
                if "Call Transcript:" in memo_text or "transcript" in memo_text.lower():
                    print(f"🎙️  CALL TRANSCRIPT FOUND!")
                    print(f"\nTranscript Preview ({len(memo_text)} chars):")
                    print(memo_text[:300] + "..." if len(memo_text) > 300 else memo_text)
                else:
                    print(f"Memo Text ({len(memo_text)} chars):")
                    print(memo_text[:200] + "..." if len(memo_text) > 200 else memo_text)
            else:
                print(f"Memo Text: (empty)")
        
        # Summary
        transcript_count = sum(1 for a in actions if "Call Transcript:" in a.get('memoText', ''))
        if transcript_count > 0:
            print(f"\n✅ Found {transcript_count} call transcript(s) attached to this ticket")
        else:
            print(f"\n⚠️  No call transcripts found in action trail")
    else:
        print(f"\n❌ Failed to retrieve actions")
        print(f"   Status: {actions_response.status_code}")
        print(f"   Response: {actions_response.text}")
    
    print("\n" + "=" * 80)
    print("\n📄 Full incident details:")
    print(json.dumps(incident_details, indent=2))
else:
    print(f"❌ Failed to retrieve incident")
    print(f"   Status: {response.status_code}")
    print(f"   Response: {response.text}")

✅ Incident details retrieved successfully!

📋 Incident: I2601 051
   ID: 05dedca1-51b6-4999-8fce-9e411689956f
   Status: Nieuw
   Brief: Wifi probleem in gebouw 3 bij Shell in Rotterdam
   Request: 08-01-2026 16:12 API_AIPilots,: 
ElevenLabs Conversation ID: **Issue Reported:** Het hele gebouw 3 van Shell in Rotterdam heeft sinds vanochtend geen wifi, wat leidt tot stilstand. **Steps Already Performed:** Geen specifieke stappen genoemd door de beller. **Steps Suggested by Agent:** Agent heeft een incident aangemaakt en zal een SMS met het incidentnummer sturen. **Next Steps Planned:** De juiste afdeling zal contact opnemen met Charles. --- Charles meldt dat het hele gebouw sinds vanochtend geen wifi heeft.
   Caller: Aalbregt, Jacob
   Category: Netwerk
   Priority: P1 (I&A)
   Created: 2026-01-08T15:12:08.000+0000
   Modified: 2026-01-08T15:12:09.000+0000


📝 ACTION TRAIL (Call Transcripts)

Retrieved 1 action(s)


--- Action 1 ---
Entry Date: 2026-01-08T15:12:09.263+0000
Person: Syst

In [12]:
# Check actions attached to a ticket (to verify transcript was added)
# This checks if a call transcript was attached via the ElevenLabs webhook

INCIDENT_NUMBER = "I2512 008"

# Get ticket ID first
response = requests.get(
    f"{TOPDESK_URL}/incidents/number/{INCIDENT_NUMBER}",
    headers=headers,
    timeout=30
)

if response.status_code == 200:
    incident = response.json()
    ticket_id = incident.get('id')
    
    print(f"✅ Ticket: {INCIDENT_NUMBER}")
    print(f"   ID: {ticket_id}")
    print(f"   Brief: {incident.get('briefDescription')}")
    print(f"   Status: {incident.get('processingStatus', {}).get('name')}\n")
    
    # Get actions for this ticket
    actions_url = f"{TOPDESK_URL}/incidents/id/{ticket_id}/actions"
    print(f"Fetching actions from: {actions_url}\n")
    print("=" * 80)
    
    actions_response = requests.get(
        actions_url,
        headers=headers,
        timeout=30
    )
    
    if actions_response.status_code == 200:
        actions = actions_response.json()
        print(f"✅ Retrieved {len(actions)} action(s)\n")
        
        if len(actions) == 0:
            print("⚠️  No actions found in action trail")
            print("\nThis ticket doesn't have any actions yet.")
            print("Call transcripts are added as actions when the ElevenLabs webhook")
            print("processes a call and creates/updates this ticket.")
        else:
            transcript_found = False
            
            for i, action in enumerate(actions, 1):
                print(f"\n{'='*80}")
                print(f"ACTION {i}")
                print(f"{'='*80}")
                print(f"Entry Date: {action.get('entryDate')}")
                
                # Safely handle person field (may be None)
                person = action.get('person')
                if person:
                    print(f"Person: {person.get('name', 'N/A')}")
                else:
                    print(f"Person: System/API")
                
                invisible = action.get('invisibleForCaller', False)
                print(f"Invisible for Caller: {invisible}")
                
                memo_text = action.get('memoText', '')
                
                # Check if this is a call transcript
                is_transcript = "Call Transcript:" in memo_text
                
                if is_transcript:
                    transcript_found = True
                    print(f"\n🎙️  CALL TRANSCRIPT DETECTED!")
                    print(f"Length: {len(memo_text)} characters")
                    print(f"\n--- Transcript Content ---")
                    # Show first 500 chars and last 200 chars if long
                    if len(memo_text) > 700:
                        print(memo_text[:500])
                        print(f"\n... ({len(memo_text) - 700} characters omitted) ...\n")
                        print(memo_text[-200:])
                    else:
                        print(memo_text)
                else:
                    # Regular action/note
                    print(f"\nMemo Text ({len(memo_text)} chars):")
                    if memo_text:
                        print(memo_text[:300] + "..." if len(memo_text) > 300 else memo_text)
                    else:
                        print("(empty)")
            
            # Summary
            print(f"\n\n{'='*80}")
            print("SUMMARY")
            print(f"{'='*80}")
            print(f"Total actions: {len(actions)}")
            
            if transcript_found:
                transcript_count = sum(1 for a in actions if "Call Transcript:" in a.get('memoText', ''))
                print(f"✅ Call transcripts found: {transcript_count}")
            else:
                print(f"⚠️  No call transcripts found in this ticket")
                print(f"\nTo add a transcript, the ElevenLabs webhook must:")
                print(f"1. Receive a post_call_transcription event")
                print(f"2. Successfully create/find this ticket")
                print(f"3. Add the transcript as an invisible action")
    else:
        print(f"❌ Failed to retrieve actions")
        print(f"   Status: {actions_response.status_code}")
        print(f"   Response: {actions_response.text}")
else:
    print(f"❌ Failed to get ticket: {response.status_code}")
    print(f"   Response: {response.text}")

✅ Ticket: I2512 008
   ID: 1386b493-4ade-4f10-8b0e-dcb5f1af0e01
   Brief: Wifi probleem in gebouw 3 bij Shell in Rotterdam
   Status: Nieuw

Fetching actions from: https://pietervanforeest-test.topdesk.net/tas/api/incidents/id/1386b493-4ade-4f10-8b0e-dcb5f1af0e01/actions

✅ Retrieved 1 action(s)


ACTION 1
Entry Date: 2025-12-01T12:50:31.233+0000
Person: System/API
Invisible for Caller: True

🎙️  CALL TRANSCRIPT DETECTED!
Length: 2023 characters

--- Transcript Content ---
Call Transcript: [00:00:00] - agent: Hoi! Je spreekt met Alex. Waarmee kan ik helpen? [00:00:06] - caller: Hallo. [00:00:07] - agent: ... [00:00:06] - caller: Ik heb een probleem met mijn wifi. [00:00:11] - agent: Oké, geen probleem! Laten we dat eens even goed bekijken. Voordat we erin duiken, wat is je volledige naam? [00:00:19] - caller: Ik heet Charles en mijn personeelsnummer is 432. [00:00:24] - agent: Hé Charles, leuk je te spreken! En je personeelsnummer is vier drie twee, duidelijk. Voo

... (1323 characters